In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from random import randint

colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
image_height = 28
image_width = 28

In [ ]:
class Dense:
    def __init__(self, units, input_size, activation, name):
        self.W = tf.Variable(tf.random.normal([units, input_size]) * tf.math.sqrt(1/input_size), name=(name + '_W'))
        self.b = tf.Variable(tf.zeros([units,1]), name=(name + '_b'))
        self.activation = activation

    def forward(self, X, training):
        Z = tf.linalg.matmul(self.W,X) + self.b
        A = self.activation(Z)
        return A

    def get_vars(self):
        return [self.W, self.b]

    def get_weigths(self):
        return [self.W]

In [ ]:
class Batch_norm:
    def __init__(self, units, input_size, activation, name):
        self.W = tf.Variable(tf.random.normal([units, input_size]) * tf.math.sqrt(1/input_size), name=(name + '_W')) 
        self.activation = activation

        self.gamma = tf.Variable(tf.ones([units, 1]), name=(name + '_gamma'))
        self.beta = tf.Variable(tf.ones([units, 1]), name=(name + '_beta'))
        self.mu_test = tf.Variable(tf.zeros([units,1]))
        self.sigma_test = tf.Variable(tf.ones([units,1]))

    def forward(self, X, training):
        e = 10**-8

        Z = tf.linalg.matmul(self.W,X)

        if training:
            mu = tf.math.reduce_mean(Z, axis=1, keepdims=True)
            sigma = tf.math.reduce_variance(Z - mu, axis=1, keepdims=True)
            sigma = tf.math.sqrt(sigma + e)

            self.mu_test.assign(0.95*self.mu_test + 0.05*mu)
            self.sigma_test.assign(0.95*self.sigma_test + 0.05*sigma)
        else:
            mu = self.mu_test
            sigma = self.sigma_test

        Z = (Z - mu) / sigma
        Z = self.gamma * Z + self.beta

        A = self.activation(Z)
        return A

    def get_vars(self):
        return [self.W, self.gamma, self.beta]

    def get_weigths(self):
        return [self.W]

In [ ]:
class Dropout:
    def __init__(self, keep_prob):
        self.keep_prob = keep_prob

    def forward(self, X, training):
        if not training:
            return X
        X = tf.nn.dropout(X, 1-self.keep_prob)
        # D = tf.random.uniform(tf.shape(X)) < self.keep_prob
        # D = tf.cast(D, dtype=tf.float32)
        # X = X * D
        # X = X / self.keep_prob
        return X

    def get_vars(self):
        return None

    def get_weigths(self):
        return None

In [ ]:
class Softmax:
    def __init__(self, classes, input_size, name):
        self.classes = classes

        self.W = tf.Variable(tf.random.normal([classes, input_size]) * tf.math.sqrt(1/input_size), name=(name + '_W'))
        self.b = tf.Variable(tf.zeros([classes,1]), name=(name + '_b'))

    def forward(self, X, training):
        Z = tf.linalg.matmul(self.W,X) + self.b

        T = tf.math.exp(Z)
        A = T / tf.math.reduce_sum(T, axis=0)
        return A

    def get_vars(self):
        return [self.W, self.b]

    def get_weigths(self):
        return [self.W]

In [ ]:
class Model:
    def __init__(self, layer_names, units, input_size, cost_function, name="model"):
        self.layers = self.__create_layers__(layer_names, units, input_size)
        self.cost = cost_function
        self.input_size = input_size
        self.name = name

    @tf.function
    def forward(self, X, training=False, testing=False):
        for layer in self.layers:
            X = layer.forward(X, training)
            if testing:
                tf.print(X)
        return X
    
    def train(self, train_ds, dev_ds, optimizer, weights_regularization=None):
        #training
        self.variables = self.__get_variables__()
        numerator = 0.
        denominator = 0.
        for X, Y in train_ds: 
            cost = self.__training_step__(X, Y, optimizer)
            m = tf.shape(X)[1]
            m = tf.cast(m, tf.float32)
            numerator = numerator + cost * m
            denominator = denominator + m
        cost_train = numerator / denominator

        #validation
        numerator = 0.
        denominator = 0.
        for X, Y in dev_ds:
            prediction = self.forward(X, training=False)
            cost = self.cost(Y, prediction) 
            m = tf.shape(X)[1]
            m = tf.cast(m, tf.float32)
            numerator = numerator + cost * m
            denominator = denominator + m
        cost_dev = numerator / denominator

        return cost_train, cost_dev

    @tf.function
    def __training_step__(self, X_train, Y_train, optimizer):
        print('Traceing...')
        with tf.GradientTape() as tape:           
            prediction = self.forward(X_train, training=True)
            cost = self.cost(Y_train, prediction)
            grads = tape.gradient(cost, self.variables)
   
        optimizer.apply_gradients(grads, self.variables)

        return cost

    def __get_weights__(self):
        weights = []
        for layer in self.layers:
            weights = layer.get_weights()
            if weights != None:
                weights += (weights)
        return weights
        
    def __get_variables__(self):
        variables = []
        for layer in self.layers:
            var = layer.get_vars()
            if var != None:
                variables += var
        return variables

    def __create_layers__(self, layer_names, units, input_size):
        units.insert(0, input_size)
        layers = []
        i = 1

        for layer_name in layer_names:
            if layer_name == 'dense':
                layer = Dense(units[i], units[i-1], tf.nn.relu, 'Dense' + str(i))
                layers.append(layer)
                i += 1

            elif layer_name == 'batch_norm':
                layer = Batch_norm(units[i], units[i-1], tf.nn.relu, 'Batch_norm' + str(i))
                layers.append(layer)
                i += 1

            elif layer_name == 'dropout':
                layer = Dropout(keep_prob)
                layers.append(layer)

            elif layer_name == 'softmax':  
                layer = Softmax(units[i], units[i-1], 'Dense' + str(i))
                layers.append(layer)
                i += 1

        return layers 

In [ ]:
class Adam:
    def __init__(self, learning_rate, beta_v = 0.9, beta_s = 0.999):
        self.v = []
        self.s = []
        self.iteration = 0
        self.learning_rate = learning_rate
        self.beta_v = beta_v
        self.beta_s = beta_s

    def apply_gradients(self, grads, variables):
        self.iteration += 1

        for i in range(len(grads)):
            if len(self.v) <= i:
                self.v.append(tf.Variable(tf.zeros(tf.shape(grads[i]))))
            if len(self.s) <= i:
                self.s.append(tf.Variable(tf.zeros(tf.shape(grads[i]))))

            self.v[i].assign(self.beta_v*self.v[i] + (1-self.beta_v)*grads[i])
            v_corrected = self.v[i] / (1 - tf.math.pow(self.beta_v,self.iteration))

            self.s[i].assign(self.beta_s*self.s[i] + (1-self.beta_s)* tf.math.square(grads[i]))
            s_corrected = self.s[i] / (1 - tf.math.pow(self.beta_s,self.iteration))

            change = self.learning_rate * (v_corrected/(tf.math.sqrt(s_corrected) + 10**-8))

            variables[i].assign_add(-change)

In [ ]:
def Loss(Y, A):
    loss = tf.math.log(A + 10**-8)
    loss = Y * loss
    loss = -tf.math.reduce_sum(loss, axis=1)
    return loss

In [ ]:
def Cost(Y, A):
    losses = Loss(Y, A)
    cost = tf.reduce_mean(losses)
    return cost

In [ ]:
def blur(img):
    img = tf.reshape(img, [1,image_height,image_width,1])
    kernel = tf.ones([3,3,1,1], dtype=tf.float32)
    mask = tf.nn.conv2d(img, kernel, strides=1, padding='SAME')
    mask = mask / 9
    img = img + mask
    img = tf.math.minimum(img, 1)
    return tf.reshape(img, [-1])

In [ ]:
def sharpen(img):
    mask = img > 0.7
    mask = tf.cast(mask, tf.float32)
    return img * mask

In [ ]:
def get_training_data(batch_size):
    (x_train, y_train), (x_dev, y_dev) = tf.keras.datasets.mnist.load_data()
    
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.map(lambda x,y: (tf.reshape(x,[-1])/255, tf.one_hot(y,10)), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_ds = train_ds.interleave(lambda x,y: tf.data.Dataset.from_tensor_slices(([x, blur(x), sharpen(x)], [y,y,y])),block_length=3, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_ds = train_ds.batch(batch_size)
    train_ds = train_ds.map(lambda x,y: (tf.linalg.matrix_transpose(x), tf.linalg.matrix_transpose(y)), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_ds = train_ds.cache()
    train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

    dev_ds = tf.data.Dataset.from_tensor_slices((x_dev, y_dev))
    dev_ds = dev_ds.map(lambda x,y: (tf.reshape(x,[-1])/255, tf.one_hot(y,10)), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dev_ds = dev_ds.interleave(lambda x,y: tf.data.Dataset.from_tensor_slices(([x, blur(x), sharpen(x)], [y,y,y])),block_length=3, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dev_ds = dev_ds.batch(batch_size)
    dev_ds = dev_ds.map(lambda x,y: (tf.linalg.matrix_transpose(x), tf.linalg.matrix_transpose(y)), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dev_ds = dev_ds.cache()
    dev_ds = dev_ds.prefetch(tf.data.experimental.AUTOTUNE)

    return train_ds, dev_ds

In [ ]:
batch_size = 512

train_ds, dev_ds = get_training_data(batch_size)

In [ ]:
layer_names = [
    ['batch_norm', 'batch_norm', 'batch_norm', 'softmax'],
    ['batch_norm', 'batch_norm', 'softmax']]
#layer_names = [['batch_norm', 'batch_norm', 'softmax']]

units = [
    [8,4,4,10],
    [8,4,10]]

names = ['4-layers', '3-layers']

input_size = 784
cost_history = []

learning_rate = 0.001

models = []
for layers, u, name in zip(layer_names, units, names):
    models.append(Model(layers, u[:], input_size, cost_function=Cost, name=name))

#optimizers = [Adam(learning_rate) for _ in range(4)]
optimizers = [Adam(learning_rate) for _ in range(len(layer_names))]
total_epochs = 0

In [ ]:
learning_rate = 0.01
for optimizer in optimizers:
    optimizer.learning_rate = learning_rate
epochs = tf.constant(100)

In [ ]:
for epoch in range(epochs):
    costs = []
    for model, optimizer in zip(models, optimizers):
        start = tf.timestamp()
        cost_train, cost_dev = model.train(train_ds, dev_ds, optimizer)
        end = tf.timestamp()
        costs.append(cost_train)
        costs.append(cost_dev)
    
    if epoch % 1 == 0:
        print('Epoch {0:4d}: cost_train={1:.7f} cost_dev={2:.7f}, time={3:.5f}'.format(total_epochs, cost_train, cost_dev,end-start))
        #print('Epoch {0:4d}     {1}'.format(total_epochs, end-start))
    total_epochs += 1

    cost_history.append(costs)

In [ ]:
plots = np.asarray(cost_history)
plots = np.transpose(plots)
m = plots.shape[1]
plots = np.reshape(plots, [-1,2,m])
for cost_train, cost_dev, model, color in zip(plots[:,0], plots[:,1], models, colors):
    i = randint(0, len(colors)-1)
    plt.plot(cost_train, color=colors[i], label=model.name+'-train')
    plt.plot(cost_dev, color=colors[i], label=model.name+'-dev', linestyle='dashed')
plt.rcParams['figure.figsize'] = [30, 10]
plt.legend()
plt.show()

In [ ]:
#calc accuracy
for model in models:
    print(model.name)
    m = 0.
    correct = 0.
    for X,Y in train_ds:
        prediction = model.forward(X)
        prediction = tf.argmax(prediction, axis=0)
        Y = tf.argmax(Y, axis=0)
        temp = prediction == Y
        temp = tf.cast(temp, tf.float32)
        correct += tf.math.reduce_sum(temp)
        m += tf.cast(tf.shape(X)[1], tf.float32)
    tf.print('Train accuracy:',correct/m)

    m = 0.
    correct = 0.
    for X,Y in dev_ds:
        prediction = model.forward(X)
        prediction = tf.argmax(prediction, axis=0)
        Y = tf.argmax(Y, axis=0)
        temp = prediction == Y
        temp = tf.cast(temp, tf.float32)
        correct += tf.math.reduce_sum(temp)
        m += tf.cast(tf.shape(X)[1], tf.float32)
    tf.print('Dev accuracy:',correct/m)
    print('_____')

In [ ]:
for X, Y in train_ds.take(1):
    prediction = models[0].forward(X)
    prediction = tf.math.argmax(prediction, axis=0)
    ims = tf.reshape(X, [image_height, image_width, -1])

offset = 10

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(ims[:,:,i+offset], cmap='binary')
    plt.xlabel(prediction[i+offset].numpy())
plt.show()

In [ ]:
f = tf.io.read_file('test.png')
img = tf.image.decode_png(f)
img = img / 255
img = tf.reshape(img, [image_height * image_width, -1])
print(img.shape)
tf.print(tf.math.reduce_max(img))
plt.imshow(tf.reshape(img,[image_height, image_width]), cmap='gray')
plt.show()

In [ ]:
for model in models:
        prediction = model.forward(img)
        tf.print('Prediction:',tf.argmax(prediction, axis=0))
        for i in prediction:
            tf.print(i)
        print('_________')